# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions

In [2]:
# create spark session
my_path = "data/sparkify_log_small.json"
spark = SparkSession \
        .builder \
        .appName("data_wrangling") \
        .getOrCreate()

users_data = spark.read.json(my_path)
users_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [3]:
users_data.describe('userId').show()

+-------+------------------+
|summary|            userId|
+-------+------------------+
|  count|             10000|
|   mean|1442.4413286423842|
| stddev| 829.8909432082613|
|    min|                  |
|    max|               999|
+-------+------------------+



In [4]:
users_data.select('page').dropDuplicates().show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [5]:
users_data.select('level').dropDuplicates().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



# Question 1

Which page did user id "" (empty string) NOT visit?

In [6]:
# TODO: write your code to answer question 1
users_data.filter(users_data.userId == "").groupby("page").count().show()

+-----+-----+
| page|count|
+-----+-----+
| Home|  187|
|About|   15|
|Login|  126|
| Help|    8|
+-----+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [7]:
# TODO: use this space to explore the behavior of the user with an empty string
empty_user = users_data.filter(users_data.userId == "")
empty_user.show()

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|            0|    null|  null| free|    null|   PUT|Login|        null|     5598|null|   307|1513721196284|     null|      |
|  null|Logged Out|     null|  null|           26|    null|  null| paid|    null|   GET| Home|        null|      428|null|   200|1513721274284|     null|      |
|  null|Logged Out|     null|  null|            5|    null|  null| free|    null|   GET| Home|        null|     2941|null|   200|1513722009284|     null|      |
|  null|Logged Out|     null|  nul

In [8]:
empty_user.select("artist").dropDuplicates().show()

+------+
|artist|
+------+
|  null|
+------+



In [9]:
for col in empty_user.columns:
    empty_user.select(col).dropDuplicates().show()

+------+
|artist|
+------+
|  null|
+------+

+----------+
|      auth|
+----------+
|Logged Out|
|     Guest|
+----------+

+---------+
|firstName|
+---------+
|     null|
+---------+

+------+
|gender|
+------+
|  null|
+------+

+-------------+
|itemInSession|
+-------------+
|           26|
|           29|
|           65|
|           19|
|            0|
|            7|
|           34|
|           57|
|           43|
|           25|
|            6|
|           58|
|            9|
|           27|
|           52|
|           17|
|           41|
|           28|
|            5|
|            1|
+-------------+
only showing top 20 rows

+--------+
|lastName|
+--------+
|    null|
+--------+

+------+
|length|
+------+
|  null|
+------+

+-----+
|level|
+-----+
| free|
| paid|
+-----+

+--------+
|location|
+--------+
|    null|
+--------+

+------+
|method|
+------+
|   PUT|
|   GET|
+------+

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+

+------------+
|registration|
+

# Question 3

How many female users do we have in the data set?

In [10]:
# TODO: write your code to answer question 3
users_data.select('gender').dropDuplicates().show()

+------+
|gender|
+------+
|     F|
|  null|
|     M|
+------+



In [11]:
users_data.groupby('gender').agg(countDistinct('userId')).show()

+------+----------------------+
|gender|count(DISTINCT userId)|
+------+----------------------+
|     F|                   462|
|  null|                     1|
|     M|                   501|
+------+----------------------+



# Question 4

How many songs were played from the most played artist?

In [12]:
# TODO: write your code to answer question 4
users_data.groupby('artist').count().sort(desc('count')).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [22]:
# TODO: write your code to answer question 5
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

function = udf(lambda ishome: int(ishome == 'Home'), IntegerType())
user_window = Window.partitionBy('userId').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

In [23]:
cusum = users_data.filter((users_data.page == 'NextSong') | (users_data.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', sum('homevisit').over(user_window))

In [24]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupby('userId','page') \
    .agg({'period': 'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
|  9.87810650887574|
+------------------+

